In [1]:
from synutility.SynIO.data_type import load_from_pickle
import networkx as nx
from math import isclose
import time
import numpy as np

from create_partitions_by_invariant import create_partitions_by_invariant
from cluster_reaction_centers import cluster_reaction_centers

In [2]:
def aggregate_charge_and_element_attributes(reaction):
    for node, attributes in reaction.nodes.data():
        attributes["element_charge"] = f"{attributes['element']}{attributes['charge']}"

    return reaction

data = load_from_pickle("C:/Users/Mark/Documents/graph_theory/ITS_graphs.pkl.gz")
reactions = [reaction["ITS"] for reaction in data]
reactions = [aggregate_charge_and_element_attributes(reaction) for reaction in reactions]

In [3]:
def edge_count_invariant(reaction_center, partition_representant_reaction_center):
    edge_count_reaction_center = len(reaction_center.edges)
    edge_count_partition_representant = len(partition_representant_reaction_center.edges)

    return edge_count_reaction_center == edge_count_partition_representant

In [4]:
def vertex_count_invariant(reaction_center, partition_representant_reaction_center):
    vertex_count_reaction_center = len(reaction_center.nodes)
    vertex_count_partition_representant = len(partition_representant_reaction_center.nodes)

    return vertex_count_reaction_center == vertex_count_partition_representant

In [5]:
# TODO: check if this implementation for degree invariant is actually correct

def vertex_degree_invariant(reaction_center, partition_representant_reaction_center):
    reaction_center_degrees = sorted([degree for node, degree in list(reaction_center.degree)])
    partition_representant_degrees = sorted([degree for node, degree in list(partition_representant_reaction_center.degree)])

    return reaction_center_degrees == partition_representant_degrees

In [6]:
def algebraic_connectivity_invariant(reaction_center, partition_representant_reaction_center):
    algebraic_connectivity_reaction_center = nx.algebraic_connectivity(reaction_center)
    algebraic_connectivity_partition_representant_reaction_center = nx.algebraic_connectivity(partition_representant_reaction_center)

    return isclose(algebraic_connectivity_reaction_center, algebraic_connectivity_partition_representant_reaction_center, rel_tol=1e-6)

In [7]:
def rank_invariant(reaction_center, partition_representant_reaction_center):
    reaction_center_rank = float(np.linalg.matrix_rank(nx.to_pandas_adjacency(reaction_center).values))
    partition_representant_rank = float(np.linalg.matrix_rank(nx.to_pandas_adjacency(partition_representant_reaction_center).values))

    return isclose(reaction_center_rank, partition_representant_rank, rel_tol=1e-6)

In [8]:
def weisfeiler_lehman_invariant(reaction_center, partition_representant_reaction_center):  
    edge_attribute = 'order'
    node_attribute = 'element_charge'

    reaction_center_hash = nx.weisfeiler_lehman_graph_hash(reaction_center, edge_attr=edge_attribute, node_attr=node_attribute, iterations = 1)
    partition_representant_hash = nx.weisfeiler_lehman_graph_hash(partition_representant_reaction_center, edge_attr=edge_attribute, node_attr=node_attribute, iterations = 1)

    return reaction_center_hash == partition_representant_hash

In [9]:
def av_length_invariant(reaction_center, partition_representant_reaction_center):
    if nx.is_connected(reaction_center) and nx.is_connected(partition_representant_reaction_center):
        avg_path_length_reaction_center = nx.average_shortest_path_length(reaction_center)
        avg_path_length_partition_representant = nx.average_shortest_path_length(partition_representant_reaction_center)

        return isclose(avg_path_length_reaction_center, avg_path_length_partition_representant, rel_tol=1e-6)
    else:
        return True

In [ ]:
def graph_diameter_invariant(reaction_center, partition_representant_reaction_center):
    if nx.is_connected(reaction_center) and nx.is_connected(partition_representant_reaction_center):
        reaction_center_diameter = nx.diameter(reaction_center)
        partition_representant_reaction_center_diameter = nx.diameter(partition_representant_reaction_center)

        return reaction_center_diameter == partition_representant_reaction_center_diameter
    else:
        return True


In [10]:
start_time = time.time()

invariant_check_function = av_length_invariant
pre_filtered_partitions = create_partitions_by_invariant(reactions, invariant_check_function)

overall_partitions = []

for pre_filtered_partition in pre_filtered_partitions:
        partitions = cluster_reaction_centers(pre_filtered_partition)
        overall_partitions = overall_partitions + partitions

end_time = time.time()

# Summary

In [ ]:
elapsed_time = end_time - start_time

print(f"Time elapsed: {elapsed_time:.2f} seconds")

print(f"Partitions found: {len(pre_filtered_partitions)}")

In [ ]:
from visualize_graph import visualize_graph

current_partition = partitions[0]

visualize_graph(current_partition[0], with_reaction_center=True)
visualize_graph(current_partition[1], with_reaction_center=True)

# multistage invariant partitioning / hierarchical clustering

In [40]:
""" invariant_config = {
    0: edge_count_invariant,
    1: vertex_degree_invariant,
    2: weisfeiler_lehman_invariant 
} """

""" invariant_config = {
    0: rank_invariant,
    1: vertex_count_invariant,
    2: edge_count_invariant,
    3: vertex_degree_invariant,
    4: weisfeiler_lehman_invariant 
} """


""" invariant_config = {
    0: vertex_count_invariant,
    1: edge_count_invariant,
    2: vertex_degree_invariant,
    3: rank_invariant,
    4: weisfeiler_lehman_invariant 
} 
 """

invariant_config = {
    0: edge_count_invariant,
}

In [ ]:
partitions = []

start_time = time.time()

for invariant_index in range(0, len(invariant_config.values())):
    invariant_check_function = invariant_config[invariant_index]

    if invariant_index == 0:
        partitions = create_partitions_by_invariant(reactions, invariant_check_function)
    else:
         for index, partition in enumerate(partitions):
            del partitions[index]
            
            sub_partitions = create_partitions_by_invariant(partition, invariant_check_function)

            partitions = partitions + sub_partitions

    print(f"{invariant_index}'s partition length = {len(partitions)}")

overall_partitions = []

for pre_filtered_partition in partitions:
    p = cluster_reaction_centers(pre_filtered_partition)
    overall_partitions = overall_partitions + p 

end_time = time.time()

- After each prepartitioning the usual nx.isomorphism check must be executed
- the goal is to optimally reduce the complete runtime by prepartitioning

In [ ]:
elapsed_time = end_time - start_time

print(f"Time elapsed: {elapsed_time:.2f} seconds")

print(f"Partitions via invariants found: {len(partitions)}")
print(f"Partitions finally found: {len(overall_partitions)}")

In [15]:
number_of_elements_per_cluster = [(index, len(p)) for index, p in enumerate(partitions)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = number_of_elements_per_cluster
sorted_data = sorted(data, key=lambda x: x[1])

original_indices_sorted, values_sorted = zip(*sorted_data)

plt.figure(figsize=(32, 20))
plt.bar(range(len(values_sorted)), values_sorted, tick_label=original_indices_sorted, color='lightgreen', edgecolor='black')
plt.xlabel('Original Index', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.title('Barplot with Values Sorted but Original Indices Shown', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
one_element_partitions = [p for p in number_of_elements_per_cluster if p[1] < 2]

len(one_element_partitions)

one_element_partitions[0]


In [ ]:
example_graph = partitions[one_element_partitions[7][0]][0]

visualize_graph(example_graph, with_reaction_center=True)

In [ ]:
two_element_partitions = [p for p in number_of_elements_per_cluster if p[1] == 2]

len(two_element_partitions)

two_element_partitions

example_graphs = partitions[two_element_partitions[0][0]]

visualize_graph(example_graphs[0], with_reaction_center=True)
visualize_graph(example_graphs[1], with_reaction_center=True)


In [ ]:
from itertools import combinations
from invariants import edge_count_invariant, vertex_count_invariant, rank_invariant, vertex_degree_invariant, global_clustering_invariant, av_length_invariant, graph_diameter_invariant, graph_girth_invariant


fns = [
    edge_count_invariant, 
    vertex_count_invariant,
    vertex_degree_invariant,
    rank_invariant,
    global_clustering_invariant,
    av_length_invariant,
    graph_diameter_invariant,
    graph_girth_invariant
]

# Generate all unique pairwise tuples
pairwise_tuples = list(combinations(fns, 2))

x = [{0: one, 1: two, 2: weisfeiler_lehman_invariant} for one, two in pairwise_tuples]


In [4]:
from own_weisfeiler_leman import weisfeiler_leman_one_iteration, hash_distribution_has_changed

def own_weisfeiler_leman_one_iteration_invariant(reaction_center, partition_representant_reaction_center):
    reaction_center_wl = weisfeiler_leman_one_iteration(reaction_center)["hash"]
    representant_center_wl = weisfeiler_leman_one_iteration(partition_representant_reaction_center)["hash"]

    return not hash_distribution_has_changed(reaction_center_wl, representant_center_wl)

In [6]:
from own_weisfeiler_leman import hash_distribution_has_changed, weisfeiler_leman_with_convergence

def own_weisfeiler_leman_convergence_invariant(reaction_center, partition_representant_reaction_center):
    reaction_center_wl = weisfeiler_leman_with_convergence(reaction_center)["hash"]
    representant_center_wl = weisfeiler_leman_with_convergence(partition_representant_reaction_center)["hash"]

    return not hash_distribution_has_changed(reaction_center_wl, representant_center_wl)

In [7]:
import time
from create_partitions_by_invariant import create_partitions_by_invariant
from cluster_reaction_centers import cluster_reaction_centers
from synutility.SynIO.data_type import load_from_pickle
from invariant_combinations import invariant_combinations
import random

from own_weisfeiler_leman import weisfeiler_leman_one_iteration, hash_distribution_has_changed


invariant_combinations = [
    {
        0: own_weisfeiler_leman_convergence_invariant
    },
]

def aggregate_charge_and_element_attributes(reaction):
    for node, attributes in reaction.nodes.data():
        attributes["element_charge"] = f"{attributes['element']}{attributes['charge']}"

    return reaction

def load_reactions():
    data = load_from_pickle("C:/Users/Mark/Documents/graph_theory/ITS_graphs.pkl.gz")
    reactions = [reaction["ITS"] for reaction in data]
    reactions = [aggregate_charge_and_element_attributes(reaction) for reaction in reactions]

    return reactions

if __name__ == "__main__":
    reactions = load_reactions()
    partitions = []

    for invariant_config_index, invariant_config in enumerate(invariant_combinations):

        complete_start_time = time.time()

        for invariant_index in range(0, len(invariant_config.values())):
            invariant_check_function = invariant_config[invariant_index]

            start_invariant = time.time()

            start_partition_assign = None
            end_partition_assign = None

            if invariant_index == 0:
                partitions = create_partitions_by_invariant(reactions, invariant_check_function)
            else:
                all_sub_partitions = []
                for index, partition in enumerate(partitions):
                    sub_partitions = create_partitions_by_invariant(partition, invariant_check_function)

                    all_sub_partitions = all_sub_partitions + sub_partitions

                partitions = all_sub_partitions

            end_invariant = time.time()
            
            print(f"{invariant_index}'s partition length = {len(partitions)}")
            print(f"{invariant_index}'s time = {(end_invariant - start_invariant):.2f}")
            
        pre_filtering_end_time = time.time()
        overall_partitions = []

        for pre_filtered_partition in partitions:
            p = cluster_reaction_centers(pre_filtered_partition)
            overall_partitions = overall_partitions + p 

        complete_end_time = time.time()

        complete_elapsed_time = complete_end_time - complete_start_time
        prefiltering_elapsed_time = pre_filtering_end_time - complete_start_time
        print(f"###Invariant Config ###")
        print(invariant_config)

        print(f"Time elapsed pre: {prefiltering_elapsed_time:.2f} seconds")
        print(f"Time elapsed complete: {complete_elapsed_time:.2f} seconds")

        print(f"Partitions via invariants found: {len(partitions)}")
        print(f"Partitions finally found: {len(overall_partitions)}")
        print("#########################################")

0's partition length = 233
0's time = 180.80
###Invariant Config ###
{0: <function own_weisfeiler_leman_convergence_invariant at 0x00000205B34BE660>}
Time elapsed pre: 180.80 seconds
Time elapsed complete: 216.72 seconds
Partitions via invariants found: 233
Partitions finally found: 299
#########################################


# Debugging

In [ ]:
clustered_partitions = cluster_reaction_centers(reactions[:10000])
len(clustered_partitions)

In [7]:
for index, partition in enumerate(clustered_partitions):
    sub_partitions = create_partitions_by_invariant(partition, own_weisfeiler_leman_invariant)

    if len(sub_partitions) != 1:
        print('diverging', index)
        divergent_partition = sub_partitions



In [ ]:
from visualize_graph import visualize_graph

divergent_partition[0][0]

In [ ]:


divergent_partition[1][0]


In [ ]:
from get_reaction_center import get_reaction_center

rc1 = get_reaction_center(reactions[14])
rc2 = get_reaction_center(reactions[540])

In [ ]:
weisfeiler_leman_one_iteration(rc1)["hash"]

In [ ]:
weisfeiler_leman_one_iteration(rc2)["hash"]

In [2]:
from invariant_combinations import invariant_combinations

invariant_combinations

[{0: <function invariants.vertex_count_invariant(reaction_center, partition_representant_reaction_center)>},
 {0: <function invariants.vertex_degree_invariant(reaction_center, partition_representant_reaction_center)>},
 {0: <function invariants.vertex_count_invariant(reaction_center, partition_representant_reaction_center)>,
  1: <function invariants.edge_count_invariant(reaction_center, partition_representant_reaction_center)>},
 {0: <function invariants.vertex_degree_invariant(reaction_center, partition_representant_reaction_center)>,
  1: <function invariants.av_length_invariant(reaction_center, partition_representant_reaction_center)>},
 {0: <function invariants.edge_count_invariant(reaction_center, partition_representant_reaction_center)>,
  1: <function invariants.vertex_degree_invariant(reaction_center, partition_representant_reaction_center)>},
 {0: <function invariants.edge_count_invariant(reaction_center, partition_representant_reaction_center)>,
  1: <function invariants.av_